In [55]:
import cv2 
import numpy as np 
if __name__ == '__main__' :
    data_path = 'image/book/'
    # Read source image.
    im1 = cv2.imread('./image/book/book1.jpg')
 
 
    # Read destination image.
    im2 = cv2.imread('./image/book/book2.jpg')

    #SIFT算法计算特征点和描述符
    sift = cv2.SIFT_create()
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params, search_params)

    gray1 = cv.cvtColor(im1, cv.COLOR_BGR2GRAY)
    kp1, ds1 = sift.detectAndCompute(gray1, None)
    gray2 = cv.cvtColor(im2, cv.COLOR_BGR2GRAY)
    kp2, ds2 = sift.detectAndCompute(gray2, None)
    #Kp表示关键点，ds表示描述子
    matches = flann.knnMatch(ds1, ds2, k=2)
 
    good = []
    for m, n in matches:
        if m.distance < 0.6* n.distance:
            good.append([m.queryIdx, m.trainIdx])
            #queryIdx是指示特征点在图像1中的索引，而trainIdx是指示特征点在图像2中的索引
       #.pt是KeyPoint类的一个属性，用于表示特征点的坐标。
    srcpoints = np.float32([kp1[m[0]].pt for m in good])#im1平面中点的坐标列数组
    dstpoints = np.float32([kp2[m[1]].pt for m in good])#im2平面中点的坐标列数组

    print("Number of matches: ", srcpoints.shape[0])
    
    
    
    good2 = []
    for m, n in matches:
        if m.distance < 0.6* n.distance:
            good2.append([m])
    
    im3 = cv2.drawMatchesKnn(im1,kp1,im2,kp2,good2,None,flags=2)
    cv2.imshow("match", im3)
    cv2.imwrite(data_path+'Match Image.png',im3)
    
    
    # Calculate Homography
    h, status = cv2.findHomography(srcpoints,dstpoints)
    print("H =\n", h)
    # Warp source image to destination based on homography
    im_out = cv2.warpPerspective(im1, h, (im2.shape[1],im2.shape[0]))
    
    # Display images
    cv2.imwrite(data_path+'Source Image.png',im1)
    cv2.imshow("Source Image", im1)
    cv2.imwrite(data_path+'Destination Image.png',im2)
    cv2.imshow("Destination Image", im2)
    cv2.imwrite(data_path+'Warped Source Image.png',im_out)
    cv2.imshow("Warped Source Image", im_out)
 
    cv2.waitKey(0)

Number of matches:  54
H =
 [[ 4.53653175e+00  3.99846911e+00 -1.99116991e+03]
 [-2.01616687e+00  5.93008851e+00 -4.94648365e+02]
 [ 2.14974400e-03  5.18833626e-03  1.00000000e+00]]
